In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://twitter.com/AlexReibman/status/1767296114267754626')

In [66]:
page_content = driver.page_source

In [28]:
soup = BeautifulSoup(page_content, 'html.parser')

In [7]:
soup.find('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu').find('span', class_='css-1qaijid r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-qvutc0 r-poiln3').text

'Diego'

In [8]:
soup.find('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu').find('div', class_='css-1rynq56 r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-18u37iz r-1wvb978').text

'@bruvseriously'

In [9]:
'https://twitter.com/' + soup.find('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu').find('a', class_='css-1rynq56 r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-xoduu5 r-1q142lx r-1w6e6rj r-9aw3ui r-3s2u2q r-1loqt21')['href']

'https://twitter.com//bruvseriously/status/1767327504795378021'

In [29]:
soup.find('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu').find('time').text

'Mar 12'

In [11]:
soup.find_all('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu')[1].find('div', class_='css-1rynq56 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-bnwqim').text

"No, it's wayyyy worse than plugins. Most GPTs are just system prompts with a coat of paint"

In [12]:
soup.find('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu').find('span', class_='css-1qaijid r-qvutc0 r-poiln3 r-n6v787 r-1cwl3u0 r-1k6nrdp r-s1qlax').text

'1'

In [32]:
soup.find('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu').find(attrs={'data-testid': 'retweet'}).text

'1'

In [13]:
# Likes
soup.find('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu').find('div', class_='css-175oi2r r-xoduu5 r-1udh08x').text

'1'

In [14]:
soup.find_all('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu')[11].find('div', class_='css-175oi2r r-1awozwy r-18u37iz r-1cmwbt1 r-1wtj0ep').text

''

In [15]:
comments_raw = []
for soup in soup.find_all('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu'):
    if soup.find('div', class_='css-175oi2r r-1awozwy r-18u37iz r-1cmwbt1 r-1wtj0ep').text == 'Ad':
        print('Ad')
    else:
        print('Not Ad')
        comments_raw.append(soup)

Not Ad
Not Ad
Not Ad
Ad
Not Ad
Not Ad
Not Ad
Not Ad
Not Ad
Not Ad
Not Ad
Not Ad
Not Ad
Ad
Not Ad
Not Ad


In [16]:
len(comments_raw)

14

In [33]:
comments = []

In [36]:
def scrape_comments(page_content):
    soup = BeautifulSoup(page_content, 'html.parser')
    comments_raw = []
    for x in soup.find_all('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu'):
        if x.find('div', class_='css-175oi2r r-1awozwy r-18u37iz r-1cmwbt1 r-1wtj0ep').text == 'Ad':
            continue
        else:
            comments_raw.append(x)
    
    comments = []
    
    for comment in comments_raw:
        
        comment_data = {}
        
        display_element = comment.find('span', class_='css-1qaijid r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-qvutc0 r-poiln3')
        if display_element:
            comment_data['Display Name'] = display_element.text
        else:
            'N/A'
            
        user_element = comment.find('div', class_='css-1rynq56 r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-18u37iz r-1wvb978')
        if user_element:
            comment_data['Username'] = user_element.text
        else:
            'N/A'
            
        url_element = comment.find('a', class_='css-1rynq56 r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-xoduu5 r-1q142lx r-1w6e6rj r-9aw3ui r-3s2u2q r-1loqt21')
        if url_element:
            comment_data['URL'] = 'https://www.twitter.com' + url_element['href']
        else:
            'N/A'
            
        time_element = comment.find('time')
        if time_element:
            comment_data['Timestamp'] = time_element.text
        else:
            'N/A'
            
        text_element = comment.find('div', class_='css-1rynq56 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-bnwqim')
        if text_element:
            comment_data['Text'] = text_element.text
        else:
            'N/A'
            
        like_element = comment.find(attrs={'data-testid': 'like'})
        if like_element:
            comment_data['No. of Likes'] = like_element.text
        else:
            'N/A'
            
        retweet_element = comment.find(attrs={'data-testid': 'retweet'})
        if retweet_element:
            comment_data['No. of Retweets'] = retweet_element.text
        else:
            'N/A'
            
        comments.append(comment_data)
        
    df = pd.DataFrame(comments)
    
    return df

In [67]:
df_temp = scrape_comments(page_content)

In [68]:
df_temp

,Display Name,Username,URL,Timestamp,Text,No. of Likes,No. of Retweets
0,Nick Dobos,@NickADobos,https://www.twitter.com/NickADobos/status/1767...,Mar 12,Dis what happens when you use bad metrics that...,6,
1,TestingCatalog News,@testingcatalog,https://www.twitter.com/testingcatalog/status/...,Mar 12,I still default to my own GPTs because use cas...,5,
2,Jordan Rains was here.,@50shadesofdrunk,https://www.twitter.com/50shadesofdrunk/status...,Mar 12,Retro fitting old processes to new tech isn't ...,1,
3,Jon Apgar,@Jonappgar,https://www.twitter.com/Jonappgar/status/17673...,Mar 12,"it's a bit embarrassing watching them ""product...",,
4,Ángel (e/acc),@Angaisb_,https://www.twitter.com/Angaisb_/status/176731...,Mar 12,Tbh I knew it would be a failure from the star...,2,
5,Dazz,@itsmrdazz,https://www.twitter.com/itsmrdazz/status/17673...,Mar 12,I've assembled an incredible team of GPTs for ...,1,
6,Nick Dobos,@NickADobos,https://www.twitter.com/NickADobos/status/1767...,2h,Release-eve ?,12,


In [69]:
df = pd.concat([df, df_temp], ignore_index=True)

In [70]:
df

,Display Name,Username,URL,Timestamp,No. of Likes,No. of Retweets,Text
0,Diego,@bruvseriously,https://www.twitter.com/bruvseriously/status/1...,Mar 12,3,1,NaN
1,Alex Reibman,@AlexReibman,https://www.twitter.com/AlexReibman/status/176...,Mar 12,11,,"No, it's wayyyy worse than plugins. Most GPTs ..."
2,Keshav Jindal,@Keshavatearth,https://www.twitter.com/Keshavatearth/status/1...,Mar 12,2,,"Haven't used the Store. Also, ""mention in chat..."
3,Kevin Piacentini,@kevinpiac,https://www.twitter.com/kevinpiac/status/17673...,Mar 12,1,,Interesting thread to follow.
4,Dallas,@i_Forget_,https://www.twitter.com/i_Forget_/status/17673...,Mar 12,,,I’ve got some good gpts i can share for people...
5,Dazz,@itsmrdazz,https://www.twitter.com/itsmrdazz/status/17673...,Mar 12,2,,I've assembled an incredible team of GPTs for ...
6,Yurii Filipchuk — e/acc,@yfilipch,https://www.twitter.com/yfilipch/status/176729...,Mar 12,1,,useless
7,Oleksandr Gamaniuk,@ogamaniuk,https://www.twitter.com/ogamaniuk/status/17677...,4h,1,,It's a failure. Here's why:\n1. Plugins always...
8,AmebaGPT,@amebagpt,https://www.twitter.com/amebagpt/status/176749...,20h,1,,I use only my own and I find them quite useful...
9,Shakir-works,@ShakirWorks,https://www.twitter.com/ShakirWorks/status/176...,Mar 12,,,The perceived value of most of the models - an...


In [71]:
out_path = 'C:\\Users\\BINARY COMPUTERS\\Documents\\Twitter.xlsx'
writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [3]:
# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the tweet
tweet_url = "https://twitter.com/NASA/status/1765051550534127788"

# Navigate to the tweet
driver.get(tweet_url)

# Wait for dynamic content to load
time.sleep(5)

In [4]:
# Scroll to load replies (example of scrolling twice)
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
time.sleep(2)
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
time.sleep(2)

In [5]:
# Extract replies (simplified example - adjust selector as needed)
replies = driver.find_elements(By.XPATH, '//div[@data-testid="reply"]')
for reply in replies:
    print(reply.text)

In [ ]:
# Clean up
driver.quit()